**Practical 3(b)**

**Feature Scaling and Dummification**

**b)Perform feature dummification to convert categorical variables into numerical representations.**

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

In [2]:
df=pd.read_csv("/content/Housing.csv")

In [3]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
one_hot = LabelBinarizer()
one_hot.fit_transform(df['furnishingstatus'])

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [5]:
one_hot.classes_

array(['furnished', 'semi-furnished', 'unfurnished'], dtype='<U14')

In [6]:
df1 = pd.get_dummies(df,columns=['mainroad', 'guestroom', 'basement', 'hotwaterheating','airconditioning',
                                 'prefarea', 'furnishingstatus'],drop_first=True)
df1.head()

,price,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,2,True,False,False,False,True,True,False,False
1,12250000,8960,4,4,4,3,True,False,False,False,True,False,False,False
2,12250000,9960,3,2,2,2,True,False,True,False,False,True,True,False
3,12215000,7500,4,2,2,3,True,False,True,False,True,True,False,False
4,11410000,7420,4,1,2,2,True,True,True,False,True,False,False,False


In [7]:
data = df['furnishingstatus'].apply(lambda x: [x])

one_hot_multiclass = MultiLabelBinarizer()
one_hot_multiclass.fit_transform(data)

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]])

In [8]:
scale_mapper = {
    "furnished": 1,
    "semi-furnished": 2,
    "unfurnished": 3}
df["furnishingstatus"].replace(scale_mapper).head()

/tmp/ipython-input-490944319.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["furnishingstatus"].replace(scale_mapper).head()


,furnishingstatus
0,1
1,1
2,2
3,1
4,1


In [9]:
from sklearn.feature_extraction import DictVectorizer

dictvectorizer = DictVectorizer(sparse=False)
df3 = df.to_dict(orient='records')
features = dictvectorizer.fit_transform(df3)
dictvectorizer.get_feature_names_out()

array(['airconditioning=no', 'airconditioning=yes', 'area', 'basement=no',
       'basement=yes', 'bathrooms', 'bedrooms',
       'furnishingstatus=furnished', 'furnishingstatus=semi-furnished',
       'furnishingstatus=unfurnished', 'guestroom=no', 'guestroom=yes',
       'hotwaterheating=no', 'hotwaterheating=yes', 'mainroad=no',
       'mainroad=yes', 'parking', 'prefarea=no', 'prefarea=yes', 'price',
       'stories'], dtype=object)

In [10]:
df4 = df.copy()
nan_mask = np.random.rand(*df4.shape) < 0.05
df4 = df4.mask(nan_mask)

In [11]:
from sklearn.impute import SimpleImputer
import numpy as np
X_complete = np.vstack((df4.values, df4.values))
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit_transform(X_complete)

array([[13300000.0, 7420.0, 4.0, ..., 2.0, 'yes', 'furnished'],
       [12250000.0, 8960.0, 3.0, ..., 3.0, 'no', 'furnished'],
       [12250000.0, 9960.0, 3.0, ..., 2.0, 'no', 'semi-furnished'],
       ...,
       [1750000.0, 3620.0, 2.0, ..., 0.0, 'no', 'unfurnished'],
       [1750000.0, 2910.0, 3.0, ..., 0.0, 'no', 'furnished'],
       [1750000.0, 3850.0, 3.0, ..., 0.0, 'no', 'unfurnished']],
      dtype=object)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
df_encoded = df.apply(LabelEncoder().fit_transform)
# use 'mainroad' as target
X = df_encoded.drop("mainroad", axis=1)
y = df_encoded["mainroad"]
# make binary target
target = np.where(y == 0, 0, 1)
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(class_weight='balanced')

In [13]:
# Indices for each class
i_class0 = np.where(y == 0)[0]
i_class1 = np.where(y == 1)[0]

# Count
n_class0 = len(i_class0)
n_class1 = len(i_class1)
# Downsample majority class
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
# Combine downsampled class1 + class0
X_down = np.vstack((X.iloc[i_class0], X.iloc[i_class1_downsampled]))
y_down = np.hstack((y.iloc[i_class0], y.iloc[i_class1_downsampled]))

print("After downsampling:", np.bincount(y_down))

After downsampling: [77 77]


In [14]:
# Upsample minority class
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

# Combine upsampled class0 + class1
X_up = np.vstack((X.iloc[i_class0_upsampled], X.iloc[i_class1]))
y_up = np.hstack((y.iloc[i_class0_upsampled], y.iloc[i_class1]))

print("After upsampling:", np.bincount(y_up))


After upsampling: [468 468]
